In [11]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

In [12]:
data_raw = pd.read_parquet("vhrdata").drop_duplicates(subset=None, keep='last', inplace=False)
drop_columns = ["ebs_Q_Charge", "ebs_Q_Discharge", "BCM_ebs_I_BATT", "dcdc_OverTemperature","IC_LowBatteryvoltage", "DCDC_realInputVoltage", "DCDC_realInputCurrent"]
data_raw = data_raw.drop(columns=drop_columns)

In [13]:
pd.DataFrame(data_raw["vin"]).drop_duplicates(subset=None, keep='first', inplace=False)

,vin
0,LM8F7D796NA013165
10,LM8F7D899NA000800
30,LM8F7D892NA001321
90,LM8F7D891NA003609
110,LM8F7D891NA000192
130,LM8F7D797MAS00719
190,LM8F7D896NA007722
230,LM8F7D897NA000312
240,LM8F7D896NA000737
270,LM8F7D996NA043361


In [16]:
data_raw[(data_raw.bms_SOC>=30.)&(data_raw.BCM_ebs_U_BATT<10.5)&(data_raw.BCM_ebs_U_BATT>5.)]["vin"].value_counts()

vin
LM8F7D991NA025057    2
LM8F7D895NA002236    2
LM8F7D792NA012370    2
Name: count, dtype: int64

In [14]:
data_raw["vcu_vehicleSpeed"].describe()

count    432629.000000
mean          6.220991
std          17.796839
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         143.887497
Name: vcu_vehicleSpeed, dtype: float64

1. ebs_Q_Charge ebs_Q_Discharge BCM_ebs_I_BATT  dcdc_OverTemperature 一直为 0.0，同时又不是状态值，直接删除这些特征
2. IC_LowBatteryvoltage  DCDC_realInputVoltage  DCDC_realInputCurrent 全为空，直接删除这些特征
3. DCDC_MCU_HVIL_Warning DCDC_BMS_HVIL_Warning DCDC_HV_InterlockStatus1 DCDC_HV_InterlockStatus dcdc_deratingStatus 作为状态值先保留
4. BCM_ebs_U_BATT 中间存在 3.0 的值将这部分提取出来进一步观察

In [ ]:
data_raw["vin"].drop_duplicates()

In [ ]:
import seaborn as sns
sns.distplot(data_raw[data_raw.dcdc_realOutputVoltage < 11.].dcdc_realOutputVoltage, hist=True, kde=False)

In [ ]:
pd.DataFrame(data_raw[data_raw.BCM_ebs_U_BATT > 14.25].BCM_ebs_U_BATT.value_counts()).sort_values(by='BCM_ebs_U_BATT')

In [ ]:
import seaborn as sns
sns.distplot(data_raw[data_raw.BCM_ebs_U_BATT > 14.252].BCM_ebs_U_BATT, hist=True, kde=False)

In [ ]:
data_raw.describe()

In [ ]:
pd.DataFrame(data_raw["BCM_ebs_U_BATT"].value_counts()).sort_values(by='BCM_ebs_U_BATT')

In [ ]:
# 清除异常数据之后的数据
data_raw = data_raw.fillna(method='ffill')
data = data_raw[(data_raw.BCM_ebs_U_BATT > 3.) & (data_raw.BCM_ebs_U_BATT < 18.)
                & (data_raw.BCM_ebs_SOC <= 100.)
                & (data_raw.bms_pLimitDrive <= 400.)
]
data.info()

In [ ]:
data["vin"].value_counts()

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt
data.BCM_ebs_U_BATT.hist(bins=10, figsize=(15,3))
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["BCM_ebs_U_BATT", "BCM_ebs_SOC"]
scatter_matrix(data[attributes], figsize=(8, 8))

In [ ]:
data[(data.bms_SOC>=30.)&(data.BCM_ebs_U_BATT<10.5)]["vin"].value_counts()

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["BCM_ebs_U_BATT", "BCM_ebs_SOC"]
scatter_matrix(data[data["vin"]=="LM8F7D895NA004066"][attributes], figsize=(8, 8))

In [ ]:


attributes = ["BCM_ebs_U_BATT",  "DCDC_AlerState"]
scatter_matrix(data[attributes], figsize=(8, 8))

In [ ]:
data.BCM_ebs_U_BATT.describe()

In [ ]:
import seaborn as sns
sns.distplot(data.bms_SOC, hist=True, kde=True)

In [ ]:
import seaborn as sns
sns.distplot(data[data.BCM_ebs_U_BATT>10.0].BCM_ebs_U_BATT, hist=True, kde=True)